# Model Training
In this step, we load the model-ready dataset and train a model including class weighting due to class imbalances. The model will be evaluated using an ROC-AUC on the validation split since this is a binary classification problem.


In [2]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

import lightgbm as lgb

# Set Path variables for data importation
CLEAN_DIR = Path("../data/clean")

# Set Path variables for exporting prediction data
OUT_VAL_PATH = CLEAN_DIR / "oof_val_preds.csv"
OUT_TEST_PATH = CLEAN_DIR / "test_predictions.csv"

# Load the datasets
train_df = pd.read_csv(CLEAN_DIR / "model_train.csv")
test_df  = pd.read_csv(CLEAN_DIR / "model_test.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print("Default rate in train (TARGET==1):", train_df["TARGET"].mean())


Train shape: (307511, 303)
Test shape: (48744, 302)
Default rate in train (TARGET==1): 0.08072881945686496


### A. Prepare the feature labels and IDs

In [3]:
TARGET_COL = "TARGET"
ID_COL = "SK_ID_CURR"

# target
y = train_df[TARGET_COL].astype(int).values

# copy feature frames
X = train_df.drop(columns=[TARGET_COL]).copy()
X_test_final = test_df.copy()

# save ids separately
id_train = X[ID_COL].values
id_test  = X_test_final[ID_COL].values

# remove ID column from model features
X = X.drop(columns=[ID_COL])
X_test_final = X_test_final.drop(columns=[ID_COL])

print("X shape:", X.shape)
print("X_test_final shape:", X_test_final.shape)


X shape: (307511, 301)
X_test_final shape: (48744, 301)


### B. Use a stratified approach to split the data

In [4]:
X_tr, X_val, y_tr, y_val, id_tr, id_val = train_test_split(
    X, y, id_train,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train split shape:", X_tr.shape)
print("Valid split shape:", X_val.shape)
print("Validation default rate:", y_val.mean())


Train split shape: (246008, 301)
Valid split shape: (61503, 301)
Validation default rate: 0.08072776937710356


### B. Train a LightGBM Model
LightGBM is an ensemble learning framework that employs gradient boosting method. It sequentially adds weak learners creating a strong learner. It works well with large datasets as this one, and is effective in memory usage. It is prefered because it leads to:
- Faster training speed and higher efficiency.

- Lower memory usage.

- Better accuracy.

- Supports of parallel, distributed, and GPU learning.

- Capable of handling large-scale data.

In [5]:
# Create a wrap data for LightGBM
lgb_train = lgb.Dataset(X_tr, label=y_tr)
lgb_valid = lgb.Dataset(X_val, label=y_val)

pos_weight = (y_tr==0).sum() / max((y_tr==1).sum(), 1)
print("scale_pos_weight =", pos_weight)

params = {
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "num_leaves": 64,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "min_data_in_leaf": 50,
    "scale_pos_weight": pos_weight,
    "verbose": -1,
}

model = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_train, lgb_valid],
    valid_names=["train","valid"],
    num_boost_round=5000,
    early_stopping_rounds=200,
    verbose_eval=100
)

print("Best iteration:", model.best_iteration)


scale_pos_weight = 11.38710976837865


TypeError: train() got an unexpected keyword argument 'early_stopping_rounds'

## 4. Validation performance (ROC-AUC + ROC curve)

In [ ]:
val_pred_proba = model.predict(X_val, num_iteration=model.best_iteration)
val_auc = roc_auc_score(y_val, val_pred_proba)
print("Validation ROC-AUC:", val_auc)

fpr, tpr, _ = roc_curve(y_val, val_pred_proba)
plt.plot(fpr, tpr)
plt.plot([0,1],[0,1],"--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title(f"ROC Curve (AUC={val_auc:.4f})")
plt.show()


## 5. Feature importance

In [ ]:
importances = model.feature_importance(importance_type="gain")
feat_imp = (
    pd.DataFrame({
        "feature": X.columns,
        "importance_gain": importances
    })
    .sort_values("importance_gain", ascending=False)
)

print(feat_imp.head(20))

plt.figure(figsize=(6,8))
feat_imp.head(25).sort_values("importance_gain").plot(
    kind="barh",
    x="feature",
    y="importance_gain"
)
plt.title("Top 25 Feature Importances (gain)")
plt.tight_layout()
plt.show()


## 6. Save validation predictions for analysis / threshold tuning

In [ ]:
CLEAN_DIR = Path("data/clean")
OUT_VAL_PATH = CLEAN_DIR / "oof_val_preds.csv"

val_results = pd.DataFrame({
    "SK_ID_CURR": id_val,
    "TARGET": y_val,
    "PRED_PROB": val_pred_proba
})
val_results.to_csv(OUT_VAL_PATH, index=False)
print("Wrote validation preds to:", OUT_VAL_PATH)

val_results.head()


## 7. Final model on FULL data + test predictions

In [ ]:
# train final model using all rows
full_lgb_train = lgb.Dataset(X, label=y)

final_model = lgb.train(
    {**params, "verbose": -1},
    full_lgb_train,
    num_boost_round=model.best_iteration  # reuse best iteration from earlier fit
)

# predict on test
test_pred_proba = final_model.predict(X_test_final)

submission_like = pd.DataFrame({
    "SK_ID_CURR": id_test,
    "PRED_PROB": test_pred_proba
})

OUT_TEST_PATH = CLEAN_DIR / "test_predictions.csv"
submission_like.to_csv(OUT_TEST_PATH, index=False)

print("Wrote test predictions to:", OUT_TEST_PATH)
submission_like.head()
